# Crawler

In [11]:
# cording: utf-8

## requests

In [ ]:
import requests

In [38]:
r = requests.get('http://52.25.113.156')

In [39]:
r.status_code == 200

True

In [40]:
contents = r.text
print contents[0:50]

<!DOCTYPE html>
<html>
<head>
  <title>Comicomu</t


## BeautifulSoup

### document
[Beautiful Soup 4.4.0 documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [ ]:
from bs4 import BeautifulSoup

In [41]:
soup = BeautifulSoup(contents, 'html.parser')

In [42]:
for link in soup.find_all('a'):
    print link.get('href')

/
/tops/about
/users/sign_up
/users/sign_in
/users/auth/twitter
/posts/5
/posts/5
/posts/4
/posts/4
/posts/3
/posts/3
/posts/1
/posts/1


In [43]:
seed = u'http://52.25.113.156'
to_crawl = {seed}
links = soup.find_all('a')
for link in links:
    url = requests.compat.urljoin(seed, link.get('href'))
    if url.find("'")!=-1: continue
    url = url.split('#')[0] # アンカーを取り除く
    if url[0:4] == 'http':
        to_crawl.add(url)
    
to_crawl

{u'http://52.25.113.156',
 u'http://52.25.113.156/',
 u'http://52.25.113.156/posts/1',
 u'http://52.25.113.156/posts/3',
 u'http://52.25.113.156/posts/4',
 u'http://52.25.113.156/posts/5',
 u'http://52.25.113.156/tops/about',
 u'http://52.25.113.156/users/auth/twitter',
 u'http://52.25.113.156/users/sign_in',
 u'http://52.25.113.156/users/sign_up'}

In [56]:
def gettextonly(soup):
    v = soup.string
    if v==None:
        c = soup.contents
        resulttext = ''
        for t in c:
            subtext = gettextonly(t)
            resulttext += subtext + '\n'
        return resulttext
    else:
        return v.strip()

textonly = gettextonly(soup)

## MeCab

In [12]:
import MeCab

In [65]:
def split_to_word(text):
    words = []
    m = MeCab.Tagger('-Ochasen')
    text = text.encode('utf-8')
    node = m.parseToNode(text)
    while node:
        words.append(node.surface)
        node = node.next
    return words

text = u"欲望が味方する Wow Wow Wow~"
for i, word in  enumerate(split_to_word(textonly)):
    print i, word

0 
1 html
2 Comicomu
3 ComiComu
4 About
5 Sign
6 up
7 Log
8 in
9 Log
10 in
11 with
12 Twitter
13 ComiComu
14 みんな
15 で
16 勉強
17 会
18 を
19 開こ
20 う
21 OSI
22 参照
23 モデル
24 勉強
25 会
26 ネットワーク
27 (
28 その
29 プロトコル
30 通信
31 )
32 の
33 理解
34 の
35 ため
36 に
37 
38 O
39 ...
40 詳細
41 1
42 人
43 が
44 参加
45 tag
46 :
47 div
48 の
49 閉じ
50 就活
51 必勝
52 法
53 面接
54 で
55 聞か
56 れる
57 事
58 とか
59 シェア
60 しよ
61 う
62 詳細
63 1
64 人
65 が
66 参加
67 tag
68 :
69 div
70 の
71 閉じ
72 asdfa
73 asdf
74 詳細
75 3
76 人
77 が
78 参加
79 tag
80 :
81 div
82 の
83 閉じ
84 JSON
85 Schema
86 を
87 書く
88 JSON
89 を
90 返す
91 RESTful
92 な
93 WEB
94 API
95 の
96 
97 仕
98 ...
99 詳細
100 1
101 人
102 が
103 参加
104 tag
105 :
106 div
107 の
108 閉じ
109 


## class Crawler

### description
オライリーの集合知プログラミングの第4章と[Pythonでつくる検索エンジン](http://nwpct1.hatenablog.com/entry/python-search-engine)を参考にしていく。   
コード規約は、[Google Python Style Guide](http://works.surgo.jp/translation/pyguide.html)をみてコーディングを意識したい。

### collection

- url_list
  - url
- word_list
  - word
- word_location
  - url_id
  - word_id
  - location
- link
  - row_id
  - from_id
  - to_id
- link_words
  - word_id
  - link_id

In [17]:
from pymongo import MongoClient
import requests
from requests.compat import urljoin
from bs4 import BeautifulSoup
import MeCab

In [66]:
class Crawler:
    # データベースの名前でクローラを初期化する
    def __init__(self, dbname):
        client = MongoClient()
        db = client[dbname]
        self.db = db

    def __del__(self):
        pass
    
    def dbcommit(self):
        pass
    
    # エントリIDを取得したり、それが存在しない場合には追加
    # するための補助関数
    def get_entry_id(self, collection, key, value, createnew=True):
        if object = self.db[collection].find({key: value}):
            return object
            object = self.db[collection].insert_one({key: value})
        return 
    
    # 個々のページをインデックスする
    def add_to_index(self, url, soup):
        if seld.is_indexed(url): return
        print 'Indexing %s' % url
        
        # 個々の単語を取得する
        text = self.get_text_only(soup)
        words = self.separate_words(text)
        
        # URL idを取得する
        url_id = self.get_entry_id('url_list', 'url', url)
        
        # それぞれの単語と、このurlのリング
        for i, word in enumerate(words):
            word_id = self.get_entry_id('word_list', 'word', word)
            self.db['word_location'].insert_one({'url_id': url_id})
    
    # HTMLのページからタグのない状態でテキストを摘出する
    def get_text_only(self, soup):
        values = soup.string
        if values == None:
            contents = soup.contents
            result_text = ''
            for t in contents:
                sub_text = self.get_text_only(t)
                result_text += sub_text + '\n'
            return result_text
        else:
            return values.strip()
    
    # 空白以外の文字で単語を分割する
    def separate_words(self, text):
        words = []
        m = MeCab.Tagger('-Ochasen')
        text = text.encode('utf-8')
        node = m.parseToNode(text)
        while node:
            words.append(node.surface)
            node = node.next
        return words
    
    # URLが既にインデックスされていたらtrueを返す
    def is_indexed(self, url):
        return False
    
    # ２つのページ間にリンクを付け加える
    def add_link_ref(self, url_from, url_to, link_text):
        pass
    
    # ページを受け取り、与えられた深さで幅優先の検索を行い
    # ページをインデクシングする
    def crawl(self, seed, max_depth=2):
        pages = {seed}
        crawled = []
        next_depth = []
        depth = 0
        while pages and depth <= max_depth:
            page_url = pages.pop()
            if page_url not in crawled:
                res = requests.get(page_url)
                try:
                    res.status_code == 200
                except:
                    print 'Could not open %s' % page
                    continue
                html_doc = res.text
                soup = BeautifulSoup(html_doc, 'html.parser')
                self.add_to_index(page_url, soup)

                for link in soup.find_all('a'):
                    link_url = urljoin(page_url, link.get('href'))
                    if link_url.find("'")!=-1: continue
                    if link_url[-1] == '/':
                        link_url = link_url[:-1] # 最後が'/'で終わる場合取り除く
                    link_url = link_url.split('#')[0] # アンカーを取り除く
                    link_url = link_url.split('?')[0] # パラメータを取り除く
                    if url[0:4] == 'http':
                        pages.add(link_url)
                    link_text = self.get_text_only(link)
                    self.add_link_ref(page_url, link_url, link_text)

                crawled.append(page_url)

            if not pages:
                pages, next_depth = next_depth, []
                depth += 1

In [67]:
crawler = Crawler('Crawler')

In [ ]:
crawler.crawl('http://52.25.113.156', 2)

In [30]:
s = 'test/'
if s[-1] == '/':
    s = s[:-1]

s

'test'

In [70]:
crawler.db['word_list']

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'Crawler'), u'word_list')